In [1]:
import numpy as np

import tvm
from tvm import relay, auto_scheduler
import tvm.relay.testing
from tvm.contrib import graph_executor

In [2]:
def get_network(name, batch_size, layout="NHWC", dtype="float32"):

    if layout == "NHWC":
        image_shape = (224, 224, 3)
    elif layout == "NCHW":
        image_shape = (3, 224, 224)
    else:
        raise ValueError("Invlida layout: "+ layout)

    input_shape = (batch_size,) + image_shape
    output_shape = (batch_size, 1000)

    if name.startswith("resnet-"):
        n_layer = int(name.split("-")[1])
        mod, params = relay.testing.resnet.get_workload(
            num_layers=n_layer,
            batch_size=batch_size,
            layout=layout,
            dtype=dtype,
            image_shape=image_shape,
        )
    elif name.startswith("resnet3d-"):
        n_layer = int(name.split("-")[1])
        mod, params = relay.testing.resnet.get_workload(
            num_layers=n_layer,
            batch_size=batch_size,
            layout=layout,
            dtype=dtype,
            image_shape=image_shape,
        )
    elif name == "mobilenet":
        mod, params = relay.testing.mobilenet.get_workload(
            batch_size=batch_size, layout=layout, dtype=dtype, image_shape=image_shape
        )
    elif name == "squeezenet_v1.1":
        assert layout == "NCHW", "squeezenet_v1.1 only supports NCHW layout"
        mod, params = relay.testing.squeezenet.get_workload(
            version="1.1",
            batch_size=batch_size,
            dtype=dtype,
            image_shape=image_shape,
        )
    elif name == "inception_v3":
        input_shape = (batch_size, 3, 299, 299) if layout == "NCHW" else (batch_size, 299, 299, 3)
        mod, params = relay.testing.inception_v3.get_workload(batch_size=batch_size, dtype=dtype)

    return mod, params, input_shape, output_shape
    
# Define the neural network and compilation target
network = "resnet-18"
batch_size = 1
layout = "NHWC"
target = tvm.target.Target("cuda")
dtype = "float32"
log_file = "%s-%s-B%d-%s.json" % (network, layout, batch_size, target.kind.name)

In [4]:
# Extract tasks from the network
print("Extract tasks...")
mod, params, input_shape, output_shape = get_network(network, batch_size, layout, dtype=dtype)
tasks, task_weights = auto_scheduler.extract_tasks(mod["main"], params, target)

for idx, task in enumerate(tasks):
    print("========== Task %d  (workload key: %s) ==========" % (idx, task.workload_key))
    print(task.compute_dag)
print(task_weights)

Extract tasks...
========== Task 0  (workload key: ["12b88bedece6984af589a28b43e0f3c4", [1, 56, 56, 64], [3, 3, 64, 128], [1, 1, 1, 128], [1, 28, 28, 128]]) ==========
placeholder = PLACEHOLDER [1, 56, 56, 64]
PaddedInput(i0, i1, i2, i3) = tir.if_then_else(((((i1 >= 1) && (i1 < 57)) && (i2 >= 1)) && (i2 < 57)), placeholder[i0, (i1 - 1), (i2 - 1), i3], 0f)
placeholder = PLACEHOLDER [3, 3, 64, 128]
Conv2dOutput(nn, yy, xx, ff) += (PaddedInput[nn, ((yy*2) + ry), ((xx*2) + rx), rc]*placeholder[ry, rx, rc, ff])
placeholder = PLACEHOLDER [1, 1, 1, 128]
T_add(ax0, ax1, ax2, ax3) = (Conv2dOutput[ax0, ax1, ax2, ax3] + placeholder[ax0, 0, 0, ax3])
T_relu(ax0, ax1, ax2, ax3) = max(T_add[ax0, ax1, ax2, ax3], 0f)

========== Task 1  (workload key: ["12b88bedece6984af589a28b43e0f3c4", [1, 14, 14, 256], [3, 3, 256, 512], [1, 1, 1, 512], [1, 7, 7, 512]]) ==========
placeholder = PLACEHOLDER [1, 14, 14, 256]
PaddedInput(i0, i1, i2, i3) = tir.if_then_else(((((i1 >= 1) && (i1 < 15)) && (i2 >= 1)) && (i2 

In [6]:
def run_tuning():
    print("Begin tuning...")
    measure_ctx = auto_scheduler.LocalRPCMeasureContext(repeat=1, min_repeat_ms=300, timeout=10)

    tuner = auto_scheduler.TaskScheduler(tasks, task_weights)
    tune_option = auto_scheduler.TuningOptions(
        num_measure_trials=200, # change this to 20000 to achieve the best performance
        runner=measure_ctx.runner,
        measure_callbacks=[auto_scheduler.RecordToFile(log_file)],
    )

    tuner.tune(tune_option)

run_tuning()

Begin tuning...
Get devices for measurement successfully!
|  ID  | Latency (ms) | Speed (GFLOPS) | Trials |----------------------------------------------------------------------

-------------------------------------------------
------------------------------  [ Task Scheduler ]
----------------------------------------------------------------------
|    0 |            - |              - |      0 |
|    1 |            - |              - |      0 |
|    2 |            - |              - |      0 |
|    3 |            - |              - |      0 |
|    4 |            - |              - |      0 |
|    5 |            - |              - |      0 |
|    6 |            - |              - |      0 |
|    7 |            - |              - |      0 |
|    8 |            - |              - |      0 |
|    9 |            - |              - |      0 |
|   10 |            - |              - |      0 |
|   11 |            - |              - |      0 |
|   12 |            - |              - |      0 |

/home/boyuan/.anaconda3/envs/tvm-build/lib/python3.8/site-packages/xgboost/training.py:17: UserWarning: Old style callback is deprecated.  See: https://xgboost.readthedocs.io/en/latest/python/callbacks.html
  warnings.warn(f'Old style callback is deprecated.  See: {link}', UserWarning)


------------------------------  [ Search ]
----------------------------------------------------------------------
Generate Sketches		#s: 1
Sample Initial Population	#s: 86	fail_ct: 1962	Time elapsed: 0.89
GA Iter: 0	Max score: 0.9899	Min score: 0.8054	#Pop: 16	#M+: 0	#M-: 0
GA Iter: 4	Max score: 1.0000	Min score: 0.9981	#Pop: 16	#M+: 1389	#M-: 0
EvolutionarySearch		#s: 16	Time elapsed: 12.59
----------------------------------------------------------------------
------------------------------  [ Measure ]
----------------------------------------------------------------------
Get 8 programs to measure:
.E.E.E.E.E.E.E.E
Time elapsed for measurement: 1.54 s
----------------------------------------------------------------------
------------------------------  [ Train cost model ]
----------------------------------------------------------------------
Time elapsed for training: 0.10 s
|  ID  | Latency (ms) | Speed (GFLOPS) | Trials |------------------------------------------------------------

[14:31:50] /home/boyuan/.anaconda3/envs/tvm-build/conda-bld/tvm-cu102-package_1636400145961/work/src/auto_scheduler/measure.cc:299: Warning: Too many errors happened during tuning. Switching to debug mode.



Time elapsed for training: |  ID  | Latency (ms) | Speed (GFLOPS) | Trials |
0.42 s
----------------------------------------------------------------------
------------------------------  [ Task Scheduler ]
----------------------------------------------------------------------
-------------------------------------------------
|    0 |            - |              - |      8 |
|    1 |            - |              - |      8 |
|    2 |            - |              - |      8 |
|    3 |            - |              - |      8 |
|    4 |            - |              - |      8 |
|    5 |            - |              - |      8 |
|    6 |            - |              - |      8 |
|    7 |            - |              - |      8 |
|    8 |            - |              - |      8 |
|    9 |            - |              - |      8 |
|   10 |            - |              - |      8 |
|   11 |            - |              - |      8 |
|   12 |            - |              - |      8 |
|   13 |            - |

...

Time elapsed for training: 0.48 s
|  ID  | Latency (ms) | Speed (GFLOPS) | Trials |
----------------------------------------------------------------------
------------------------------  [ -------------------------------------------------
|    0 |            - |              - |      8 |
|    1 |            - |              - |      8 |
|    2 |            - |              - |      8 |
|    3 |            - |              - |      8 |
|    4 |            - |              - |      8 |
|    5 |            - |              - |      8 |
|    6 |            - |              - |      8 |
|    7 |            - |              - |      8 |
|    8 |            - |              - |      8 |
|    9 |            - |              - |      8 |
|   10 |            - |              - |      8 |
|   11 |            - |              - |      8 |
|   12 |            - |              - |      8 |
|   13 |            - |              - |      8 |
|   14 |            - |              - |      8 |
|   15 |   

[14:34:02] /home/boyuan/.anaconda3/envs/tvm-build/conda-bld/tvm-cu102-package_1636400145961/work/src/auto_scheduler/measure.cc:299: Warning: Too many errors happened during tuning. Switching to debug mode.



Time elapsed for training: |  ID  | Latency (ms) | Speed (GFLOPS) | Trials |
-------------------------------------------------
|    0 |            - |              - |      8 |
|    1 |            - |              - |      8 |
|    2 |            - |              - |      8 |
|    3 |            - |              - |      8 |
|    4 |            - |              - |      8 |
|    5 |            - |              - |      8 |
|    6 |            - |              - |      8 |
|    7 |            - |              - |      8 |
|    8 |            - |              - |      8 |
|    9 |            - |              - |      8 |
|   10 |            - |              - |      8 |
|   11 |            - |              - |      8 |
|   12 |            - |              - |      8 |
|   13 |            - |              - |      8 |
|   14 |            - |              - |      8 |
|   15 |            - |              - |      8 |
|   16 |            - |              - |      8 |0.52 s
-----------------

[14:34:39] /home/boyuan/.anaconda3/envs/tvm-build/conda-bld/tvm-cu102-package_1636400145961/work/src/auto_scheduler/measure.cc:299: Warning: Too many errors happened during tuning. Switching to debug mode.



Time elapsed for training: 0.51 s


In [9]:
# Official tutorial cannot run.

# Compile with the history best
print("Compile...")
with auto_scheduler.ApplyHistoryBest(log_file):
    with tvm.transform.PassContext(opt_level=3, config={"relay.backend.use_auto_scheduler": True}):
        lib = relay.build(mod, target=target, params=params)

# Create graph executor
dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))
data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype(dtype))
module.set_input("data", data_tvm)

# Evaluate
print("Evaluate inference time cost...")
print(module.benchmark(dev, repeat=3, min_repeat_ms=500))

Compile...


TVMError: Traceback (most recent call last):
  12: TVMFuncCall
  11: _ZNSt17_Function_handlerIFvN
  10: tvm::relay::backend::RelayBuildModule::GetFunction(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#3}::operator()(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*) const
  9: tvm::relay::backend::RelayBuildModule::BuildRelay(tvm::IRModule, std::unordered_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, tvm::runtime::NDArray, std::hash<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, std::equal_to<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > >, std::allocator<std::pair<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const, tvm::runtime::NDArray> > > const&, tvm::runtime::String)
  8: tvm::build(tvm